In [1]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
from SimLib import config_sim as conf
from SimLib import sipm_mapping as DAQ
import sys
#sys.path.append("/home/viherbos/GITHUB/PETALO_analysis")
import fit_library
import scipy.signal as sc
import itertools as it
import multiprocessing as mp
import time
#import TOF_library as T_lib
%matplotlib nbagg
%reload_ext autoreload
%autoreload 2   


In [2]:
class ENERGY_compute(object):

    def __init__(self, path, file_name, SIPM, Matrix_O):
        self.p_name = path
        self.f_name = file_name
        os.chdir(path)
        self.tof_wave = np.array(pd.read_hdf(file_name,key='MC/tof_waveforms'),
                                 dtype='int')
        self.Matrix_O   = Matrix_O
        self.ring_dim   = Matrix_O.shape
        self.n_sipms    = SIPM['n_sipms']
        self.first_sipm = SIPM['first_sipm']
        self.part_acc   = []

    def __call__(self,event):
        try:
            event_select    = np.argwhere(self.tof_wave[:,0]==event)
            event_tof       = self.tof_wave[event_select[:,0],1:]
            event_tof[:,0]  = event_tof[:,0]*-1-self.first_sipm
            # SiPM  |  Timebin  |  Charge
            time_length = np.max(event_tof[:,1])

            pe_table = np.zeros((time_length+1,self.n_sipms))

            for i in range(event_tof.shape[0]):
                pe_table[event_tof[i,1],event_tof[i,0]] = event_tof[i,2]

            part_acc  = np.sum(pe_table,axis=0)
            ev1_sipm  = np.argmax(part_acc)
            ev1_coord = np.where(self.Matrix_O == ev1_sipm)
            # Roll SiPM Matrixes to find opposite side of detector
            Xe = np.roll(self.Matrix_O,-ev1_coord[1]+self.ring_dim[1]//4,axis=1)
            # Select opposite side of detector
            Xe_sel = Xe[:,self.ring_dim[1]//2:]
            Xe_sel_1D = Xe_sel.reshape(-1)
            # Select first side of detector
            Xd_sel = Xe[:,0:self.ring_dim[1]//2]
            Xd_sel_1D = Xd_sel.reshape(-1)

            Xe_ener = np.sum(part_acc[Xe_sel_1D])
            Xd_ener = np.sum(part_acc[Xd_sel_1D])

            # print("E1=%f - E2=%f" % (Xe_ener,Xd_ener))
            # print("EVENT_n: %d" % event)
        except:
            Xe_ener = 0
            Xd_ener = 0
        return np.array([Xe_ener, Xd_ener])



class TOF_compute(object):
    """ SIPM : [risetime_tau (ps), falltime_tau (ps)]
    """
    def __init__(self, path, file_name, SIPM, Matrix_O, time_window, TE_TDC, TE_E, time_bin=5):
        self.p_name = path
        self.f_name = file_name
        # SPE response computation
        self.n_sipms    = SIPM['n_sipms']
        self.first_sipm = SIPM['first_sipm']
        
        self.Matrix_O = Matrix_O
        self.time_window = time_window
        self.TE_TDC = TE_TDC
        self.TE_E = TE_E
        self.time_bin = time_bin
        
        
        tau_sipm   = SIPM['tau_sipm']
        time     = np.arange(0,80000,time_bin)
        alfa = 1.0/tau_sipm[1]
        beta = 1.0/tau_sipm[0]
        t_p = np.log(beta/alfa)/(beta-alfa)
        K = (beta)*np.exp(alfa*t_p)/(beta-alfa)
        self.spe_resp = K*(np.exp(-time*alfa)-np.exp(-time*beta))

        os.chdir(path)
        
        self.tof_wave   = np.array(pd.read_hdf(file_name,key='MC/tof_waveforms'),
                                   dtype='int')

    def convolve_tof(self,signal,spe_resp):
        conv_first = np.hstack([spe_resp,np.zeros(len(signal)-1)])
        conv_res   = np.zeros(len(signal)+len(spe_resp)-1)
        pe_pos = np.argwhere(signal > 0)
        pe_recov = signal[pe_pos]
        for i in range(len(pe_recov)):
            desp = np.roll(conv_first,pe_pos[i])*pe_recov[i]
            conv_res = desp + conv_res
        return conv_res

    
    def TDC_first_photon(self, event):
        

        event_select    = np.argwhere(self.tof_wave[:,0]==event)
        event_tof       = self.tof_wave[event_select[:,0],1:]
        event_tof[:,0]  = event_tof[:,0]*-1-self.first_sipm

        # Beware of empty events
        timestamp_v = np.zeros(self.n_sipms)
        try:         
            
            time_length = np.max(event_tof[:,1])
            #pe_table = np.zeros((time_length+1,self.n_sipms))
            pe_table = np.zeros(self.n_sipms)
            for i in range(event_tof.shape[0]):
                if event_tof[i,1]<timestamp_v[event_tof[i,0]] or timestamp_v[event_tof[i,0]]==0:
                    timestamp_v[event_tof[i,0]] = event_tof[i,1]
                pe_table[event_tof[i,0]] += event_tof[i,2]
            self.part_acc  = pe_table
            # SiPM  |  Timebin  |  Charge
            
            
        except:
            self.part_acc  = np.zeros(self.n_sipms)
        
        return timestamp_v
    
    
    def TDC_convolution(self,event):
        event_select    = np.argwhere(self.tof_wave[:,0]==event)
        event_tof       = self.tof_wave[event_select[:,0],1:]
        event_tof[:,0]  = event_tof[:,0]*-1-self.first_sipm
        # SiPM  |  Timebin  |  Charge

        # Beware of Empty Events
        # try:
        #time_length = np.max(event_tof[:,1])
        #print ("Problema: %d" % time_length)
        pe_table = np.zeros((self.time_window,self.n_sipms))

        for i in range(event_tof.shape[0]):
            if event_tof[i,1] < self.time_window:
                pe_table[event_tof[i,1],event_tof[i,0]] = event_tof[i,2]

        #if time_window == -1:
        #    self.conv_table = np.zeros((np.max(event_tof[:,1])+1 + self.spe_resp.shape[0]-1, self.n_sipms))
        #else:
        self.conv_table = np.zeros((pe_table.shape[0] + self.spe_resp.shape[0]-1, self.n_sipms))

        for i in range(self.n_sipms):
            if not np.all(pe_table[:,i]==0): #np.max(pe_table[:,i])>0:
                self.conv_table[:,i] = self.convolve_tof(pe_table[0:self.time_window,i],self.spe_resp)
                #conv_table[:,i] = np.convolve(pe_table[:,i],self.spe_resp)
                #conv_table[:,i] = sc.fftconvolve(pe_table[:,i],self.spe_resp)

        #charge_acc     = np.cumsum(self.conv_table,axis=0)
        #self.part_acc  = np.cumsum(pe_table,axis=0)

        timestamp_v = np.array([])
        for i in range(self.conv_table.shape[1]):
            timestamp  = np.argwhere(self.conv_table[0:self.time_window,i]>self.TE_TDC)
            # We take only the first part up to time_window to speed up the computation
            if timestamp.size == 0:
                timestamp = 0
            else:
                timestamp  = np.min(timestamp)

            timestamp_v = np.hstack([timestamp_v,timestamp])

        self.part_acc = np.zeros(self.n_sipms)
        for i in range(event_tof.shape[0]):
            self.part_acc[event_tof[i,0]] += event_tof[i,2]

        #except:
           # timestamp_v      = np.zeros(self.n_sipms)
           # self.part_acc    = np.zeros(self.n_sipms)
           # self.conv_table  = np.zeros((1,self.n_sipms))
            
        return timestamp_v


    def __call__(self, event, method):
        t_stamp_v = np.array([]).reshape(0,self.n_sipms)
        ring_dim  = self.Matrix_O.shape
        TOF       = np.array([])
        j=0
        
        
        if method=="first_photon":
            timestamp = self.TDC_first_photon(event)
        else:
            timestamp = self.TDC_convolution(event)
        t_stamp_v = np.vstack([t_stamp_v,timestamp])
       

        #print ("Processing Event : %d" % event)
        
        timestamp_M    = np.ma.MaskedArray(timestamp,timestamp<1)
        gamma1_sipm    = np.ma.argmin(timestamp_M)
        gamma1_tdc     = np.ma.min(timestamp_M)
        gamma2_sipm    = np.zeros(gamma1_sipm.shape)
        gamma2_tdc     = np.zeros(gamma1_sipm.shape)

        gamma1_coord = np.where(self.Matrix_O==gamma1_sipm)
        
        # Roll SiPM Matrixes to find opposite side of detector
        Xe = np.roll(self.Matrix_O,-gamma1_coord[1]+ring_dim[1]//4,axis=1)
        # Select opposite side of detector
        Xe_sel = Xe[:,ring_dim[1]//2:]
        Xe_sel_1D = Xe_sel.reshape(-1)

        # Select first side of detector
        Xd_sel = Xe[:,0:ring_dim[1]//2]
        Xd_sel_1D = Xd_sel.reshape(-1)

        Xe_ener = np.sum(self.part_acc[Xe_sel_1D])
        Xd_ener = np.sum(self.part_acc[Xd_sel_1D])

        try:
            OPO_g = timestamp_M[Xe_sel_1D]
            gamma2_tdc = np.ma.min(OPO_g)
            gamma2_coord = Xe_sel_1D[np.ma.argmin(OPO_g)]
        except:
            gamma2_tdc = 0

        # Get rid of singles
        TOF_p = (gamma1_tdc - gamma2_tdc)/2.0

        selec_cond = np.logical_not(np.isnan(TOF_p)) and (Xd_ener>self.TE_E[0]) and (Xd_ener<self.TE_E[1]) \
                     and (Xe_ener>self.TE_E[0]) and (Xe_ener<self.TE_E[1])

        if selec_cond:
            TOF = TOF_p
            print("SiPM1 = %d | SiPM2 = %d | TOF = %f" % (gamma1_sipm, gamma2_coord, TOF_p))
        else:
            TOF = -10000

        return TOF

# Configuration Reading

In [3]:
path         = "/volumedisk0/home/viherbos/DAQ_data/"
jsonfilename = "CUBE"
SIM_CONT=conf.SIM_DATA(filename=path+jsonfilename+".json",read=True)
data = SIM_CONT.data
L1_Slice, Matrix_I, Matrix_O, topo = DAQ.SiPM_Mapping(data,data['L1']['map_style'])


Number of SiPM : 3500 
Number of ASICS : 55 
Minimum Number of L1 : 6 
Available ASICS = 55
Connected ASICS = 55
Instanciated L1 = 6
L1 number 0 has 9 ASICs
L1 number 1 has 9 ASICs
L1 number 2 has 9 ASICs
L1 number 3 has 10 ASICs
L1 number 4 has 9 ASICs
L1 number 5 has 9 ASICs


# Data & SiPM config

TDC = TOF_compute("/volumedisk0/home/paolafer/vicente/",name+"."+str(i).zfill(3)+".pet.h5",
                                     SIPM        = SIPM,
                                     Matrix_O    = Matrix_O,
                                     time_window = 10000,
                                     TE_TDC      = 1,
                                     TE_E        = [1000,1600],
                                     time_bin    = time_bin)

def TOF_comp_wrapper(args):
    return TDC(*args)

TOF_comp_wrapper([10,"convolution"])

In [4]:
SIPM = {'n_sipms':3500, 'first_sipm':1000, 'tau_sipm':[100,15000]}
# Rise time constant limited by external elements (cabling, ASCI impedance)
# 100 ps is a rough estimation based on crosstalk limit due to feedtrough design
# Fall time constant is realistic, based on bibliography and measurements

TE_range = [0.25,0.5,0.75,1,1.5,2,2.5,3]
time_bin = 5
TOF_TE_TDC = []

name = "petit_ring_tof_high_stat"

for j in range(len(TE_range)):
    TOF = np.array([])
    start_time = time.time()
    #############################
    for i in range(4):
        TDC = TOF_compute("/volumedisk0/home/paolafer/vicente/",name+"."+str(i).zfill(3)+".pet.h5",
                                         SIPM        = SIPM,
                                         Matrix_O    = Matrix_O,
                                         time_window = 10000,
                                         TE_TDC      = TE_range[j],
                                         TE_E        = [1000,1600],
                                         time_bin    = time_bin)

        # Multiprocessing

        def TOF_comp_wrapper(args):
            return TDC(*args)

        TE = 1   #np.arange(1,4,0.5)

        #TOF_comp_wrapper(100)

        pool_size = mp.cpu_count() 
        pool = mp.Pool(processes=12)
        pool_output = pool.map(TOF_comp_wrapper, zip([i for i in range(0,25000)],it.repeat("conv")))                                                    
        #time_window, TE_TDC, ev_range, TE_E
        pool.close()
        pool.join()

        # Introduce a random sign to symmetrize distribution
        random_sign = (np.random.rand(len(pool_output))>0.5)*-1
        random_sign = random_sign + random_sign + 1
        TOF = np.hstack([TOF, np.array(pool_output) * random_sign])


    print("---- %s SECONDS FOR TE_TDC = %s------" % (time.time()-start_time, TE_range[j]))
    #################################
        
    TOF_TE_TDC.append(TOF) 

Opening petit_ring_tof_high_stat.000.pet.h5 in read-only mode
SiPM1 = 1196 | SiPM2 = 2153 | TOF = -14.000000
SiPM1 = 1420 | SiPM2 = 2907 | TOF = -3.500000
SiPM1 = 800 | SiPM2 = 1936 | TOF = -7.000000
SiPM1 = 72 | SiPM2 = 2434 | TOF = -25.000000
SiPM1 = 517 | SiPM2 = 2527 | TOF = -7.500000
SiPM1 = 2371 | SiPM2 = 1411 | TOF = -5.000000
SiPM1 = 2355 | SiPM2 = 1917 | TOF = -4.000000
SiPM1 = 2967 | SiPM2 = 1832 | TOF = -6.000000
SiPM1 = 3177 | SiPM2 = 990 | TOF = -7.500000
SiPM1 = 2987 | SiPM2 = 977 | TOF = -9.000000
SiPM1 = 1176 | SiPM2 = 1785 | TOF = -5.000000
SiPM1 = 2385 | SiPM2 = 1247 | TOF = -8.500000
SiPM1 = 2345 | SiPM2 = 1389 | TOF = -19.000000
SiPM1 = 2649 | SiPM2 = 1691 | TOF = -11.500000
SiPM1 = 2209 | SiPM2 = 897 | TOF = -6.500000
SiPM1 = 2004 | SiPM2 = 866 | TOF = -1.500000
SiPM1 = 3216 | SiPM2 = 505 | TOF = -9.000000
SiPM1 = 2430 | SiPM2 = 943 | TOF = -16.500000
SiPM1 = 830 | SiPM2 = 2845 | TOF = -14.500000
SiPM1 = 2037 | SiPM2 = 1430 | TOF = -21.500000
SiPM1 = 1331 | SiPM2 =

Process ForkPoolWorker-44:
  File "/software/anaconda3/envs/IC3.5/lib/python3.5/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
Process ForkPoolWorker-46:
Process ForkPoolWorker-38:
Process ForkPoolWorker-43:
Process ForkPoolWorker-47:
Traceback (most recent call last):
Process ForkPoolWorker-48:
Process ForkPoolWorker-45:
Process ForkPoolWorker-40:
Process ForkPoolWorker-39:
Traceback (most recent call last):
Process ForkPoolWorker-42:
Process ForkPoolWorker-41:
Process ForkPoolWorker-37:
Traceback (most recent call last):
  File "/software/anaconda3/envs/IC3.5/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/software/anaconda3/envs/IC3.5/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/software/anaconda3/envs/IC3.5/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._arg

KeyboardInterrupt: 

In [ ]:
fit = fit_library.gauss_fit()
fig3 = plt.figure() #figsize=(10,8))

#for i in range(len(TE_range)): 
data = TOF #pool_output[i]
data = time_bin*data[(data>-500)*(data<500)]
fit(data,'auto')
i = 0
fit.plot(axis = fig3.add_subplot(111),
     title = "Time of Flight TE = "+str(TE_range[i])+"pe",
     xlabel = "Time Stamp in picoseconds",
     ylabel = "Hits",
     res=False, 
     fit=True)
plt.tight_layout()
print("Valid Events = %d" % (data.shape))

os.chdir("/volumedisk0/home/viherbos/DAQ_data/")
    with pd.HDFStore("TOF.h5",complevel=9, complib='bzip2') as store:
        TE_range = pd.DataFrame(data=TE_range)
        TOF_data = pd.DataFrame(data=TOF_TE_TDC))        
        store.put('TE_range',TE_range)
        store.put('TOF_data',TOF_data)
        store.close()


# Convolution Test

#timestamp, accu_table, conv_table = TDC.TDC(25,-1,1)   # Event, Time Window for TDC, TE_TDC
#print("%d" % np.min(timestamp[np.argwhere(timestamp>0)]))
fig = plt.figure()
plt.plot(TDC.conv_table[:,2729])
np.argmax(TDC.part_acc)



time = 20000
first_SiPM = 1000
phi_o = np.zeros(Matrix_O.shape)
# Data Matrix Composition
for i in range(Matrix_O.shape[0]):
    for j in range(Matrix_O.shape[1]):
        phi_o[i,j] = accu_table[time,int(Matrix_O[i,j])]

fig = plt.figure(figsize=(10,1.5))
plt.imshow(phi_o)
plt.show()
print(np.min(timestamp[np.argwhere(timestamp>0)])*5)

# Energy Computation Test

Energy = T_lib.ENERGY_compute("/volumedisk0/home/viherbos/DAQ_data/","petit_ring_tof_high_stat.pet.h5",SIPM,Matrix_O)
Energy(100)

# Energy Computation with Multiprocessing

# Multiprocessing
Energy = T_lib.ENERGY_compute("/volumedisk0/home/viherbos/DAQ_data/","petit_ring_tof_high_stat.pet.h5",SIPM,Matrix_O)   

def ENER_comp_wrapper(args):   
    return Energy(args)

pool_size = mp.cpu_count() 
pool = mp.Pool(processes=12)
#pool_output = pool.map(DAQ_map, [i for i in L1_Slice])
pool_output = pool.map(ENER_comp_wrapper, [i for i in range(0,25000)])
pool.close()
pool.join()



Energy_v = np.array([pool_output[i] for i in range(len(pool_output))]).reshape(-1)

fig2 = plt.figure()
fit = fit_library.gauss_fit()
data = Energy_v[Energy_v>100]
fit(data,50)
fit.plot(axis = fig2.add_subplot(111),
     title = "Energy Spectrum (Singles)",
     xlabel = "Photons",
     ylabel = "Hits",
     res=False, 
     fit=True)

In [ ]:
mp.cpu_count()

In [ ]:
E(110)